In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("input"))

# Any results you write to the current directory are saved as output.
import gc
gc.enable()

['proc_test.csv.gz', 'proc_train.csv.gz', 'test_id.csv']


# Model, and First Level Results Loading

> First level results generation took long.. so I dump the intermediate result and use it here

In [ ]:
usecols = [
    'date_block_num',
    'shop_id',
    'item_id',
    'item_category_id',
    'cum_shop_num',
    'cum_item_num',
    'cum_item_cat',
    'cum_item_sales',
    'exp_mean_enc_shop_item_target',
    'exp_mean_enc_shop_item_target_std',
    'exp_mean_enc_shop_item_trans_days',
    'exp_mean_enc_shop_item_price_med',
    'exp_mean_enc_shop_item_price_mean',
    'exp_mean_enc_shop_item_price_std',
    'exp_mean_enc_shop_target',
    'exp_mean_enc_shop_target_med',
    'exp_mean_enc_shop_target_mean',
    'exp_mean_enc_shop_target_std',
    'exp_mean_enc_shop_price_med',
    'exp_mean_enc_shop_price_mean',
    'exp_mean_enc_shop_price_std',
    'exp_mean_enc_item_target',
    'exp_mean_enc_item_target_med',
    'exp_mean_enc_item_target_mean',
    'exp_mean_enc_item_target_std',
    'exp_mean_enc_item_price_med',
    'exp_mean_enc_item_price_mean',
    'exp_mean_enc_item_price_std',
    'target'
]
train = pd.read_csv('input/proc_train.csv.gz', usecols=usecols)
test = pd.read_csv('input/proc_test.csv.gz', usecols=usecols)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train_dates = train.date_block_num.values
test_dates = test.date_block_num.values
train_y = train.target.values
drop_cols = ['date_block_num', 'target']

In [ ]:
train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)
gc.collect()

In [ ]:
mask = train_dates >= 12
train = train[mask].reset_index(drop=True)
train_y = train_y[mask]
train_dates = train_dates[mask]
print(pd.Series(train_dates).value_counts())

In [ ]:
merge = pd.concat([train, test], ignore_index=True)
merge_dates = np.concatenate([train_dates, test_dates], axis=0)

merge_y = np.concatenate([train_y, np.zeros((len(test_dates),))], axis=0)
print(merge_y.shape)

stage2_train_dates = merge_dates[merge_dates>=level2_date_block[0]]
print(pd.Series(stage2_train_dates).value_counts())
stage2_train_mask = (stage2_train_dates <= level2_date_block[-1])
stage2_valid_mask = (stage2_train_dates == 33)
stage2_final_train_mask = (stage2_train_dates <= 33)
stage2_test_mask = (stage2_train_dates==34)
stage2_train_y = merge_y[merge_dates>=level2_date_block[0]]

print(pd.Series(stage2_train_mask).value_counts())
print(pd.Series(stage2_valid_mask).value_counts())
print(pd.Series(stage2_final_train_mask).value_counts())
print(pd.Series(stage2_test_mask).value_counts())

In [ ]:
import pickle
with open('stage2_data.dataframe', 'rb') as handle:
    stage2_df = pickle.load(handle)
    print(stage2_df.shape)
    print(type(stage2_df))
    
with open('stage2_data_text.dataframe', 'rb') as handle:
    stage2_data_text = pickle.load(handle)
    print(stage2_data_text.shape)
    print(type(stage2_data_text))

In [ ]:
stage2_df.loc[:, 'rg_text'] = stage2_data_text

In [ ]:
stage2_df.head(5)

In [ ]:
model_cols = ['rg', 'lasso', 'rf', 'lgbm', 'knn', 'rg_text']
#stage2_df = stage2_df[model_cols]
stage2_df = stage2_df[['rg', 'lasso', 'rf', 'lgbm', 'knn', 'rg_text', 'kmean_dist_label']]
stage2_df[model_cols].corr()

In [ ]:
from scipy.stats.mstats import hmean, gmean

In [ ]:
def stage2_feat_engineer(df, model_cols=[]):
    model_cnt = df[model_cols].shape[1]
    
    print('Calculating 2 g\\h\\m-mean')
    for i in range(model_cnt-1): # i = 0 to model_cnt-2
        for j in range(i+1, model_cnt): # j = i+1 to model_cnt-1
            cols = [model_cols[i], model_cols[j]]
            df.loc[:, 'gmean_' + str(i) + '_'+ str(j)] = gmean(df.loc[:,cols].values, axis=1)
            df.loc[:, 'mean_' + str(i) + '_'+ str(j)] = np.mean(df.loc[:,cols].values, axis=1)

    if model_cnt > 3:
        print('Calculating 3 g\\h\\m-mean')        
        for i in range(model_cnt-2): # i = 0 to model_cnt-3
            for j in range(i+1, model_cnt-1): # j = i+1 to model_cnt-2
                for k in range(j+1, model_cnt): # k = j+1 to model_cnt-1
                    cols = [model_cols[i], model_cols[j], model_cols[k]]
                    df.loc[:, 'gmean_' + str(i) + '_'+ str(j) + '_'+ str(k)] = gmean(df.loc[:,cols].values, axis=1)
                    df.loc[:, 'mean_' + str(i) + '_'+ str(j) + '_'+ str(k)] = np.mean(df.loc[:,cols].values, axis=1)

    print('Calculating all g\\h\\m-mean')             
    df.loc[:, 'gmean_all'] = gmean(df.loc[:,model_cols].values, axis=1)
    df.loc[:, 'mean'] = np.mean(df.loc[:,model_cols].values, axis=1)
    df.loc[:, 'med'] = np.median(df.loc[:,model_cols].values, axis=1)
    df.loc[:, 'max'] = np.amax(df.loc[:,model_cols].values, axis=1)
    df.loc[:, 'min'] = np.amin(df.loc[:,model_cols].values, axis=1)
    df.loc[:, 'std'] = np.std(df.loc[:,model_cols].values, axis=1)

In [ ]:
stage2_feat_engineer(stage2_df, model_cols)
stage2_df.head()

In [ ]:
stage2_df = pd.concat([stage2_df, merge[merge_dates>=level2_date_block[0]].reset_index(drop=True)], axis=1)
print(stage2_df.shape)
stage2_df.head(3)

# Generate Submission

In [ ]:
import lightgbm as lgb

In [ ]:
'''
lgbm = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.075, n_estimators=80,
                         max_depth=6, min_split_gain=0.0, min_child_samples=40,
                         colsample_bytree=.95, subsample=.9,
                         reg_alpha=.0, reg_lambda=.4,    
                         random_state=0, n_jobs =4)
'''
with open('lgb_stage2.model', 'rb') as handle:
    lgbm = pickle.load(handle)
    print('lgb model loaded')    

In [ ]:
test_ID = pd.read_csv('input/test_id.csv')
print(test_ID.shape)
test_ID.head()

In [ ]:
val_pred = lgbm.predict(stage2_df[stage2_final_train_mask].values)
val_pred = np.clip(val_pred, 0., 20.)
print('Train rmse=', mean_squared_error(stage2_train_y[stage2_final_train_mask], val_pred)**.5)

lgbm_test_pred = lgbm.predict(stage2_df[stage2_test_mask].values)
lgbm_test_pred = np.clip(lgbm_test_pred, 0., 20.)
test_ID.loc[:, 'item_cnt_month'] = lgbm_test_pred

test_ID.to_csv('lgb_submission.csv', index=False)